In [ ]:
# Complete Stock Market Time Series Forecasting Project
# ALL REQUIRED MODELS: ARIMA, SARIMA, Prophet, LSTM

# Install required packages
!pip install yfinance prophet plotly streamlit statsmodels tensorflow scikit-learn -q

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

# For ARIMA/SARIMA models
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

# For LSTM model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

class CompleteStockForecaster:
    def __init__(self, symbol='AAPL', period='2y'):
        self.symbol = symbol
        self.period = period
        self.data = None
        self.models = {}
        self.forecasts = {}
        self.metrics = {}

    def fetch_data(self):
        """Fetch stock data using yfinance"""
        print(f"📈 Fetching data for {self.symbol}...")
        ticker = yf.Ticker(self.symbol)
        self.data = ticker.history(period=self.period)
        print(f"✅ Data fetched: {len(self.data)} records")
        print(f"📅 Date range: {self.data.index[0].strftime('%Y-%m-%d')} to {self.data.index[-1].strftime('%Y-%m-%d')}")
        return self.data

    def analyze_time_series(self):
        """Analyze time series properties"""
        print("\n🔍 TIME SERIES ANALYSIS:")

        # Check stationarity
        result = adfuller(self.data['Close'].dropna())
        print(f"📊 Stationarity Test (ADF):")
        print(f"   - ADF Statistic: {result[0]:.4f}")
        print(f"   - p-value: {result[1]:.4f}")
        print(f"   - Is Stationary: {'Yes' if result[1] < 0.05 else 'No'}")

        # Seasonal decomposition (use shorter period that fits our data)
        period = min(50, len(self.data['Close']) // 3)  # Use 50 days or 1/3 of data, whichever is smaller
        if len(self.data['Close']) >= 2 * period:
            decomposition = seasonal_decompose(self.data['Close'], model='multiplicative', period=period)
            print(f"📈 Seasonal decomposition completed with period={period}")
        else:
            print(f"⚠  Insufficient data for seasonal decomposition (need {2*period}, have {len(self.data['Close'])})")
            decomposition = None

        return decomposition

    def prepare_data_differencing(self, data, d=1):
        """Make data stationary for ARIMA"""
        diff_data = data.copy()
        for i in range(d):
            diff_data = diff_data.diff().dropna()
        return diff_data

    # MODEL 1: ARIMA
    def train_arima(self, order=(5,1,0)):
        """Train ARIMA model"""
        print(f"\n🔄 Training ARIMA{order} model...")

        try:
            # Make data stationary
            stationary_data = self.prepare_data_differencing(self.data['Close'], d=order[1])

            # Fit ARIMA model
            model = ARIMA(self.data['Close'], order=order)
            fitted_model = model.fit()

            self.models['ARIMA'] = fitted_model
            print(f"✅ ARIMA model trained successfully!")
            print(f"   - AIC: {fitted_model.aic:.2f}")

            return fitted_model

        except Exception as e:
            print(f"❌ ARIMA training failed: {e}")
            return None

    # MODEL 2: SARIMA
    def train_sarima(self, order=(1,1,1), seasonal_order=(1,1,1,12)):
        """Train SARIMA model"""
        print(f"\n🔄 Training SARIMA{order}x{seasonal_order} model...")

        try:
            model = SARIMAX(self.data['Close'],
                           order=order,
                           seasonal_order=seasonal_order)
            fitted_model = model.fit(disp=False)

            self.models['SARIMA'] = fitted_model
            print(f"✅ SARIMA model trained successfully!")
            print(f"   - AIC: {fitted_model.aic:.2f}")

            return fitted_model

        except Exception as e:
            print(f"❌ SARIMA training failed: {e}")
            return None

    # MODEL 3: Prophet
    def train_prophet(self):
        """Train Facebook Prophet model"""
        print(f"\n🔄 Training Prophet model...")

        try:
            # Prepare data
            df = self.data.reset_index()
            df = df[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})
            df['ds'] = df['ds'].dt.tz_localize(None)  # Remove timezone

            # Train model
            model = Prophet(
                daily_seasonality=True,
                weekly_seasonality=True,
                yearly_seasonality=True,
                changepoint_prior_scale=0.05
            )
            model.fit(df)

            self.models['Prophet'] = {'model': model, 'data': df}
            print(f"✅ Prophet model trained successfully!")

            return model

        except Exception as e:
            print(f"❌ Prophet training failed: {e}")
            return None

    # MODEL 4: LSTM
    def prepare_lstm_data(self, lookback=60):
        """Prepare data for LSTM model"""
        # Scale the data
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(self.data['Close'].values.reshape(-1, 1))

        # Create sequences
        X, y = [], []
        for i in range(lookback, len(scaled_data)):
            X.append(scaled_data[i-lookback:i, 0])
            y.append(scaled_data[i, 0])

        X, y = np.array(X), np.array(y)
        X = X.reshape((X.shape[0], X.shape[1], 1))

        return X, y, scaler

    def train_lstm(self, lookback=60, epochs=50):
        """Train LSTM model"""
        print(f"\n🔄 Training LSTM model...")

        try:
            # Prepare data
            X, y, scaler = self.prepare_lstm_data(lookback)

            # Split data
            train_size = int(len(X) * 0.8)
            X_train, X_test = X[:train_size], X[train_size:]
            y_train, y_test = y[:train_size], y[train_size:]

            # Build LSTM model
            model = Sequential([
                LSTM(50, return_sequences=True, input_shape=(lookback, 1)),
                Dropout(0.2),
                LSTM(50, return_sequences=True),
                Dropout(0.2),
                LSTM(50),
                Dropout(0.2),
                Dense(1)
            ])

            model.compile(optimizer='adam', loss='mean_squared_error')

            # Train model
            history = model.fit(X_train, y_train,
                              epochs=epochs,
                              batch_size=32,
                              validation_data=(X_test, y_test),
                              verbose=0)

            self.models['LSTM'] = {
                'model': model,
                'scaler': scaler,
                'lookback': lookback,
                'history': history
            }

            print(f"✅ LSTM model trained successfully!")
            print(f"   - Final Loss: {history.history['loss'][-1]:.6f}")

            return model

        except Exception as e:
            print(f"❌ LSTM training failed: {e}")
            return None

    def forecast_all_models(self, days=30):
        """Generate forecasts from all models"""
        print(f"\n🔮 Generating {days}-day forecasts...")

        forecasts = {}

        # ARIMA Forecast
        if 'ARIMA' in self.models and self.models['ARIMA'] is not None:
            try:
                forecast = self.models['ARIMA'].forecast(steps=days)
                forecasts['ARIMA'] = forecast
                print("✅ ARIMA forecast generated")
            except Exception as e:
                print(f"❌ ARIMA forecast failed: {e}")

        # SARIMA Forecast
        if 'SARIMA' in self.models and self.models['SARIMA'] is not None:
            try:
                forecast = self.models['SARIMA'].forecast(steps=days)
                forecasts['SARIMA'] = forecast
                print("✅ SARIMA forecast generated")
            except Exception as e:
                print(f"❌ SARIMA forecast failed: {e}")

        # Prophet Forecast
        if 'Prophet' in self.models and self.models['Prophet'] is not None:
            try:
                model = self.models['Prophet']['model']
                future = model.make_future_dataframe(periods=days)
                forecast = model.predict(future)
                forecasts['Prophet'] = forecast.tail(days)['yhat'].values
                print("✅ Prophet forecast generated")
            except Exception as e:
                print(f"❌ Prophet forecast failed: {e}")

        # LSTM Forecast
        if 'LSTM' in self.models and self.models['LSTM'] is not None:
            try:
                lstm_info = self.models['LSTM']
                model = lstm_info['model']
                scaler = lstm_info['scaler']
                lookback = lstm_info['lookback']

                # Get last sequence
                last_sequence = scaler.transform(self.data['Close'].tail(lookback).values.reshape(-1, 1))

                predictions = []
                current_sequence = last_sequence.reshape(1, lookback, 1)

                for _ in range(days):
                    pred = model.predict(current_sequence, verbose=0)
                    predictions.append(pred[0, 0])

                    # Update sequence
                    current_sequence = np.roll(current_sequence, -1, axis=1)
                    current_sequence[0, -1, 0] = pred[0, 0]

                # Inverse transform
                predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
                forecasts['LSTM'] = predictions
                print("✅ LSTM forecast generated")

            except Exception as e:
                print(f"❌ LSTM forecast failed: {e}")

        self.forecasts = forecasts
        return forecasts

    def evaluate_models(self):
        """Evaluate all models"""
        print(f"\n📊 EVALUATING MODEL PERFORMANCE...")

        metrics = {}
        actual_prices = self.data['Close'].tail(50).values  # Last 50 days for evaluation

        for model_name, model in self.models.items():
            if model is None:
                continue

            try:
                if model_name == 'ARIMA':
                    fitted_values = model.fittedvalues.tail(50).values
                elif model_name == 'SARIMA':
                    fitted_values = model.fittedvalues.tail(50).values
                elif model_name == 'Prophet':
                    # Get fitted values from Prophet
                    df = model['data']
                    prophet_model = model['model']
                    fitted = prophet_model.predict(df)
                    fitted_values = fitted['yhat'].tail(50).values
                elif model_name == 'LSTM':
                    # Evaluate LSTM on test data
                    X, y, scaler = self.prepare_lstm_data()
                    test_start = int(len(X) * 0.8)
                    X_test = X[test_start:]
                    y_test = y[test_start:]

                    predictions = model['model'].predict(X_test, verbose=0)
                    fitted_values = scaler.inverse_transform(predictions).flatten()
                    actual_prices = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

                # Calculate metrics
                mape = mean_absolute_percentage_error(actual_prices, fitted_values) * 100
                rmse = np.sqrt(mean_squared_error(actual_prices, fitted_values))

                metrics[model_name] = {
                    'MAPE': round(mape, 2),
                    'RMSE': round(rmse, 2)
                }

                print(f"✅ {model_name}: MAPE={mape:.2f}%, RMSE={rmse:.2f}")

            except Exception as e:
                print(f"❌ {model_name} evaluation failed: {e}")
                metrics[model_name] = {'MAPE': 'N/A', 'RMSE': 'N/A'}

        self.metrics = metrics
        return metrics

    def plot_comprehensive_results(self, days=30):
        """Create comprehensive visualization"""
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Historical Data & Forecasts',
                'Model Performance Comparison',
                'Forecast Comparison',
                'Time Series Decomposition'
            ),
            specs=[[{"colspan": 2}, None],
                   [{"type": "bar"}, {"type": "scatter"}]]
        )

        # Historical data
        dates = self.data.index
        fig.add_trace(
            go.Scatter(x=dates, y=self.data['Close'],
                      name='Historical', line=dict(color='blue')),
            row=1, col=1
        )

        # Forecasts
        colors = {'ARIMA': 'red', 'SARIMA': 'green', 'Prophet': 'purple', 'LSTM': 'orange'}
        future_dates = pd.date_range(start=dates[-1] + pd.Timedelta(days=1), periods=days)

        for model_name, forecast in self.forecasts.items():
            if forecast is not None:
                fig.add_trace(
                    go.Scatter(x=future_dates, y=forecast,
                              name=f'{model_name} Forecast',
                              line=dict(color=colors.get(model_name, 'black'), dash='dash')),
                    row=1, col=1
                )

        # Model performance bar chart
        if self.metrics:
            models = list(self.metrics.keys())
            mapes = [self.metrics[m]['MAPE'] if self.metrics[m]['MAPE'] != 'N/A' else 0 for m in models]

            fig.add_trace(
                go.Bar(x=models, y=mapes, name='MAPE (%)', marker_color='lightblue'),
                row=2, col=1
            )

        # Forecast values comparison
        if self.forecasts:
            for model_name, forecast in self.forecasts.items():
                if forecast is not None:
                    fig.add_trace(
                        go.Scatter(x=list(range(1, len(forecast)+1)), y=forecast,
                                  name=f'{model_name}', mode='lines+markers'),
                        row=2, col=2
                    )

        fig.update_layout(height=800, title_text=f"Complete Stock Analysis: {self.symbol}")
        fig.show()

    def generate_comprehensive_report(self):
        """Generate detailed analysis report"""
        current_price = self.data['Close'].iloc[-1]

        report = f"""
        📈 COMPREHENSIVE STOCK MARKET ANALYSIS REPORT
        {'='*60}

        🏢 STOCK INFORMATION:
        - Symbol: {self.symbol}
        - Current Price: ${current_price:.2f}
        - Analysis Period: {self.period}
        - Data Points: {len(self.data)} records
        - Date Range: {self.data.index[0].strftime('%Y-%m-%d')} to {self.data.index[-1].strftime('%Y-%m-%d')}

        🤖 MODELS IMPLEMENTED:
        """

        for model_name in ['ARIMA', 'SARIMA', 'Prophet', 'LSTM']:
            status = "✅ Trained" if model_name in self.models and self.models[model_name] is not None else "❌ Failed"
            report += f"        - {model_name}: {status}\n"

        report += f"""
        📊 MODEL PERFORMANCE METRICS:
        """

        for model_name, metrics in self.metrics.items():
            report += f"        - {model_name}: MAPE={metrics['MAPE']}%, RMSE={metrics['RMSE']}\n"

        if self.forecasts:
            report += f"""
        🔮 FORECAST SUMMARY (30 days):
        """
            for model_name, forecast in self.forecasts.items():
                if forecast is not None and len(forecast) > 0:
                    forecast_price = forecast[-1]
                    change = ((forecast_price - current_price) / current_price) * 100
                    report += f"        - {model_name}: ${forecast_price:.2f} ({change:+.2f}%)\n"

        # Find best model
        best_model = None
        best_mape = float('inf')

        for model_name, metrics in self.metrics.items():
            if metrics['MAPE'] != 'N/A' and metrics['MAPE'] < best_mape:
                best_mape = metrics['MAPE']
                best_model = model_name

        report += f"""
        🏆 BEST PERFORMING MODEL: {best_model} (MAPE: {best_mape}%)

        📋 TECHNICAL ANALYSIS SUMMARY:
        - Trend: {'Upward' if self.data['Close'].iloc[-1] > self.data['Close'].iloc[-30] else 'Downward'}
        - Volatility: {((self.data['Close'].std() / self.data['Close'].mean()) * 100):.2f}%
        - Volume Trend: {self.data['Volume'].tail(10).mean()/1000000:.2f}M avg (last 10 days)

        ⚠  DISCLAIMER:
        This analysis is for educational purposes only.
        Not financial advice. Past performance doesn't guarantee future results.
        Always consult with financial professionals before making investment decisions.
        """

        return report

# MAIN EXECUTION
if __name__ == "__main__":
    print("🚀 STARTING COMPLETE STOCK MARKET ANALYSIS...")
    print("=" * 60)

    # Initialize forecaster
    forecaster = CompleteStockForecaster('AAPL', '2y')

    # Step 1: Data Collection & Analysis
    data = forecaster.fetch_data()
    decomposition = forecaster.analyze_time_series()

    # Step 2: Train All Models
    print("\n TRAINING ALL REQUIRED MODELS:")
    print("-" * 40)

    # Train each model
    forecaster.train_arima(order=(5,1,0))
    forecaster.train_sarima(order=(1,1,1), seasonal_order=(1,1,1,12))
    forecaster.train_prophet()
    forecaster.train_lstm(epochs=20)  # Reduced epochs for faster training

    # Step 3: Generate Forecasts
    forecasts = forecaster.forecast_all_models(days=30)

    # Step 4: Evaluate Models
    metrics = forecaster.evaluate_models()

    # Step 5: Visualize Results
    forecaster.plot_comprehensive_results()

    # Step 6: Generate Report
    report = forecaster.generate_comprehensive_report()
    print(report)

    # Save results
    results_df = pd.DataFrame({
        'Model': list(forecasts.keys()),
        'Forecast_30_days': [forecasts[model][-1] if len(forecasts[model]) > 0 else None
                            for model in forecasts.keys()],
        'MAPE': [metrics.get(model, {}).get('MAPE', 'N/A') for model in forecasts.keys()],
        'RMSE': [metrics.get(model, {}).get('RMSE', 'N/A') for model in forecasts.keys()]
    })

    results_df.to_csv(f'{forecaster.symbol}_complete_analysis.csv', index=False)

    print(f"\n Complete analysis saved to {forecaster.symbol}_complete_analysis.csv")
    print("\n ALL ANALYSIS COMPLETE! Ready for presentation.")
    print("\n DELIVERABLES GENERATED:")
    print("   ✓ Historical data analysis")
    print("   ✓ ARIMA model implementation")
    print("   ✓ SARIMA model implementation")
    print("   ✓ Prophet model implementation")
    print("   ✓ LSTM model implementation")
    print("   ✓ Model comparison & evaluation")
    print("   ✓ Interactive visualizations")
    print("   ✓ Comprehensive report")
    print("   ✓ CSV results export")

🚀 STARTING COMPLETE STOCK MARKET ANALYSIS...
📈 Fetching data for AAPL...
✅ Data fetched: 500 records
📅 Date range: 2023-09-05 to 2025-09-02

🔍 TIME SERIES ANALYSIS:
📊 Stationarity Test (ADF):
   - ADF Statistic: -1.8366
   - p-value: 0.3624
   - Is Stationary: No
📈 Seasonal decomposition completed with period=50

 TRAINING ALL REQUIRED MODELS:
----------------------------------------

🔄 Training ARIMA(5, 1, 0) model...
✅ ARIMA model trained successfully!
   - AIC: 2670.48

🔄 Training SARIMA(1, 1, 1)x(1, 1, 1, 12) model...
✅ SARIMA model trained successfully!
   - AIC: 2666.02

🔄 Training Prophet model...


DEBUG:cmdstanpy:input tempfile: /tmp/tmpyz6m1t_o/3qpz3cif.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpyz6m1t_o/i6vly37m.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=42574', 'data', 'file=/tmp/tmpyz6m1t_o/3qpz3cif.json', 'init=/tmp/tmpyz6m1t_o/i6vly37m.json', 'output', 'file=/tmp/tmpyz6m1t_o/prophet_model5179q9vs/prophet_model-20250902144116.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:41:16 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:41:17 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


✅ Prophet model trained successfully!

🔄 Training LSTM model...
✅ LSTM model trained successfully!
   - Final Loss: 0.012480

🔮 Generating 30-day forecasts...
✅ ARIMA forecast generated
✅ SARIMA forecast generated
✅ Prophet forecast generated
✅ LSTM forecast generated

📊 EVALUATING MODEL PERFORMANCE...
✅ ARIMA: MAPE=0.92%, RMSE=2.81
✅ SARIMA: MAPE=0.92%, RMSE=2.95
✅ Prophet: MAPE=2.51%, RMSE=6.64
✅ LSTM: MAPE=3.23%, RMSE=8.85


KeyError: -1

In [ ]:
# Complete Stock Market Time Series Forecasting Project - FIXED VERSION
# ALL REQUIRED MODELS: ARIMA, SARIMA, Prophet, LSTM

# Install required packages
!pip install yfinance prophet plotly streamlit statsmodels tensorflow scikit-learn -q

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

# For ARIMA/SARIMA models
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

# For LSTM model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

class CompleteStockForecaster:
    def __init__(self, symbol='AAPL', period='2y'):
        self.symbol = symbol
        self.period = period
        self.data = None
        self.models = {}
        self.forecasts = {}
        self.metrics = {}

    def fetch_data(self):
        """Fetch stock data using yfinance with error handling"""
        print(f"📈 Fetching data for {self.symbol}...")
        try:
            ticker = yf.Ticker(self.symbol)
            self.data = ticker.history(period=self.period)

            # Check if data is empty
            if self.data.empty:
                raise ValueError(f"No data found for symbol {self.symbol}")

            # Clean the data
            self.data = self.data.dropna()

            if len(self.data) < 50:
                raise ValueError(f"Insufficient data: only {len(self.data)} records found")

            print(f"✅ Data fetched: {len(self.data)} records")
            print(f"📅 Date range: {self.data.index[0].strftime('%Y-%m-%d')} to {self.data.index[-1].strftime('%Y-%m-%d')}")
            return self.data

        except Exception as e:
            print(f"❌ Error fetching data: {e}")
            return None

    def analyze_time_series(self):
        """Analyze time series properties with error handling"""
        if self.data is None or self.data.empty:
            print("❌ No data available for analysis")
            return None

        print("\n🔍 TIME SERIES ANALYSIS:")

        try:
            # Check stationarity
            close_data = self.data['Close'].dropna()
            if len(close_data) < 10:
                print("❌ Insufficient data for stationarity test")
                return None

            result = adfuller(close_data)
            print(f"📊 Stationarity Test (ADF):")
            print(f"   - ADF Statistic: {result[0]:.4f}")
            print(f"   - p-value: {result[1]:.4f}")
            print(f"   - Is Stationary: {'Yes' if result[1] < 0.05 else 'No'}")

            # Seasonal decomposition with proper error handling
            min_periods = 14  # Minimum periods for decomposition
            if len(close_data) >= 3 * min_periods:
                try:
                    decomposition = seasonal_decompose(close_data, model='multiplicative', period=min_periods)
                    print(f"📈 Seasonal decomposition completed with period={min_periods}")
                    return decomposition
                except Exception as e:
                    print(f"⚠  Seasonal decomposition failed: {e}")
            else:
                print(f"⚠  Insufficient data for seasonal decomposition (need {3*min_periods}, have {len(close_data)})")

            return None

        except Exception as e:
            print(f"❌ Time series analysis failed: {e}")
            return None

    def prepare_data_differencing(self, data, d=1):
        """Make data stationary for ARIMA with error handling"""
        try:
            diff_data = data.dropna().copy()
            for i in range(d):
                diff_data = diff_data.diff().dropna()
            return diff_data
        except Exception as e:
            print(f"❌ Data differencing failed: {e}")
            return None

    # MODEL 1: ARIMA
    def train_arima(self, order=(2,1,2)):
        """Train ARIMA model with improved error handling"""
        print(f"\n🔄 Training ARIMA{order} model...")

        if self.data is None or len(self.data) < 50:
            print("❌ Insufficient data for ARIMA training")
            return None

        try:
            # Use a more conservative order for better stability
            close_data = self.data['Close'].dropna()

            # Fit ARIMA model with error handling
            model = ARIMA(close_data, order=order)
            fitted_model = model.fit()

            self.models['ARIMA'] = fitted_model
            print(f"✅ ARIMA model trained successfully!")
            print(f"   - AIC: {fitted_model.aic:.2f}")

            return fitted_model

        except Exception as e:
            print(f"❌ ARIMA training failed: {e}")
            # Try simpler model
            try:
                print("🔄 Trying simpler ARIMA(1,1,1)...")
                model = ARIMA(close_data, order=(1,1,1))
                fitted_model = model.fit()
                self.models['ARIMA'] = fitted_model
                print(f"✅ ARIMA(1,1,1) model trained successfully!")
                return fitted_model
            except Exception as e2:
                print(f"❌ Simple ARIMA also failed: {e2}")
                return None

    # MODEL 2: SARIMA
    def train_sarima(self, order=(1,1,1), seasonal_order=(1,1,1,7)):
        """Train SARIMA model with improved parameters"""
        print(f"\n🔄 Training SARIMA{order}x{seasonal_order} model...")

        if self.data is None or len(self.data) < 50:
            print("❌ Insufficient data for SARIMA training")
            return None

        try:
            close_data = self.data['Close'].dropna()

            # Use weekly seasonality (7 days) which is more appropriate for stock data
            model = SARIMAX(close_data,
                           order=order,
                           seasonal_order=seasonal_order,
                           enforce_stationarity=False,
                           enforce_invertibility=False)
            fitted_model = model.fit(disp=False, maxiter=100)

            self.models['SARIMA'] = fitted_model
            print(f"✅ SARIMA model trained successfully!")
            print(f"   - AIC: {fitted_model.aic:.2f}")

            return fitted_model

        except Exception as e:
            print(f"❌ SARIMA training failed: {e}")
            # Try without seasonal component
            try:
                print("🔄 Trying SARIMA without seasonal component...")
                model = SARIMAX(close_data, order=order)
                fitted_model = model.fit(disp=False)
                self.models['SARIMA'] = fitted_model
                print(f"✅ SARIMA (no seasonal) trained successfully!")
                return fitted_model
            except Exception as e2:
                print(f"❌ SARIMA without seasonal also failed: {e2}")
                return None

    # MODEL 3: Prophet
    def train_prophet(self):
        """Train Facebook Prophet model with error handling"""
        print(f"\n🔄 Training Prophet model...")

        if self.data is None or len(self.data) < 50:
            print("❌ Insufficient data for Prophet training")
            return None

        try:
            # Prepare data
            df = self.data.reset_index()
            df = df[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})

            # Handle timezone issues
            if df['ds'].dt.tz is not None:
                df['ds'] = df['ds'].dt.tz_localize(None)

            # Remove any remaining NaN values
            df = df.dropna()

            if len(df) < 10:
                print("❌ Insufficient clean data for Prophet")
                return None

            # Train model with more conservative parameters
            model = Prophet(
                daily_seasonality=True,
                weekly_seasonality=True,
                yearly_seasonality=False,  # Disabled for shorter timeframes
                changepoint_prior_scale=0.01,  # More conservative
                seasonality_prior_scale=0.1,
                uncertainty_samples=100
            )
            model.fit(df)

            self.models['Prophet'] = {'model': model, 'data': df}
            print(f"✅ Prophet model trained successfully!")

            return model

        except Exception as e:
            print(f"❌ Prophet training failed: {e}")
            return None

    # MODEL 4: LSTM
    def prepare_lstm_data(self, lookback=30):  # Reduced lookback for shorter datasets
        """Prepare data for LSTM model with error handling"""
        try:
            if self.data is None or len(self.data) < lookback + 10:
                print(f"❌ Insufficient data for LSTM (need {lookback + 10}, have {len(self.data) if self.data is not None else 0})")
                return None, None, None

            # Scale the data
            scaler = MinMaxScaler(feature_range=(0, 1))
            close_data = self.data['Close'].dropna().values
            scaled_data = scaler.fit_transform(close_data.reshape(-1, 1))

            # Create sequences
            X, y = [], []
            for i in range(lookback, len(scaled_data)):
                X.append(scaled_data[i-lookback:i, 0])
                y.append(scaled_data[i, 0])

            if len(X) == 0:
                print("❌ No sequences created for LSTM")
                return None, None, None

            X, y = np.array(X), np.array(y)
            X = X.reshape((X.shape[0], X.shape[1], 1))

            return X, y, scaler

        except Exception as e:
            print(f"❌ LSTM data preparation failed: {e}")
            return None, None, None

    def train_lstm(self, lookback=30, epochs=20):
        """Train LSTM model with improved error handling"""
        print(f"\n🔄 Training LSTM model...")

        try:
            # Prepare data
            X, y, scaler = self.prepare_lstm_data(lookback)
            if X is None:
                return None

            # Split data
            train_size = max(int(len(X) * 0.8), 1)
            X_train, X_test = X[:train_size], X[train_size:]
            y_train, y_test = y[:train_size], y[train_size:]

            # Build simpler LSTM model for better stability
            model = Sequential([
                LSTM(32, return_sequences=True, input_shape=(lookback, 1)),
                Dropout(0.2),
                LSTM(32),
                Dropout(0.2),
                Dense(1)
            ])

            model.compile(optimizer='adam', loss='mean_squared_error')

            # Train model with validation only if we have test data
            if len(X_test) > 0:
                history = model.fit(X_train, y_train,
                                  epochs=epochs,
                                  batch_size=min(16, len(X_train)),
                                  validation_data=(X_test, y_test),
                                  verbose=0)
            else:
                history = model.fit(X_train, y_train,
                                  epochs=epochs,
                                  batch_size=min(16, len(X_train)),
                                  verbose=0)

            self.models['LSTM'] = {
                'model': model,
                'scaler': scaler,
                'lookback': lookback,
                'history': history
            }

            print(f"✅ LSTM model trained successfully!")
            print(f"   - Final Loss: {history.history['loss'][-1]:.6f}")

            return model

        except Exception as e:
            print(f"❌ LSTM training failed: {e}")
            return None

    def forecast_all_models(self, days=30):
        """Generate forecasts from all models with error handling"""
        print(f"\n🔮 Generating {days}-day forecasts...")

        forecasts = {}

        # ARIMA Forecast
        if 'ARIMA' in self.models and self.models['ARIMA'] is not None:
            try:
                forecast = self.models['ARIMA'].forecast(steps=days)
                forecasts['ARIMA'] = forecast.values if hasattr(forecast, 'values') else forecast
                print("✅ ARIMA forecast generated")
            except Exception as e:
                print(f"❌ ARIMA forecast failed: {e}")

        # SARIMA Forecast
        if 'SARIMA' in self.models and self.models['SARIMA'] is not None:
            try:
                forecast = self.models['SARIMA'].forecast(steps=days)
                forecasts['SARIMA'] = forecast.values if hasattr(forecast, 'values') else forecast
                print("✅ SARIMA forecast generated")
            except Exception as e:
                print(f"❌ SARIMA forecast failed: {e}")

        # Prophet Forecast
        if 'Prophet' in self.models and self.models['Prophet'] is not None:
            try:
                model = self.models['Prophet']['model']
                future = model.make_future_dataframe(periods=days)
                forecast = model.predict(future)
                forecasts['Prophet'] = forecast.tail(days)['yhat'].values
                print("✅ Prophet forecast generated")
            except Exception as e:
                print(f"❌ Prophet forecast failed: {e}")

        # LSTM Forecast
        if 'LSTM' in self.models and self.models['LSTM'] is not None:
            try:
                lstm_info = self.models['LSTM']
                model = lstm_info['model']
                scaler = lstm_info['scaler']
                lookback = lstm_info['lookback']

                # Get last sequence
                close_data = self.data['Close'].dropna()
                if len(close_data) >= lookback:
                    last_sequence = scaler.transform(close_data.tail(lookback).values.reshape(-1, 1))

                    predictions = []
                    current_sequence = last_sequence.reshape(1, lookback, 1)

                    for _ in range(days):
                        pred = model.predict(current_sequence, verbose=0)
                        predictions.append(pred[0, 0])

                        # Update sequence
                        current_sequence = np.roll(current_sequence, -1, axis=1)
                        current_sequence[0, -1, 0] = pred[0, 0]

                    # Inverse transform
                    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
                    forecasts['LSTM'] = predictions
                    print("✅ LSTM forecast generated")
                else:
                    print("❌ Insufficient data for LSTM forecast")

            except Exception as e:
                print(f"❌ LSTM forecast failed: {e}")

        self.forecasts = forecasts
        return forecasts

    def evaluate_models(self):
        """Evaluate all models with improved error handling"""
        print(f"\n📊 EVALUATING MODEL PERFORMANCE...")

        metrics = {}

        if self.data is None or len(self.data) < 20:
            print("❌ Insufficient data for model evaluation")
            return metrics

        # Use last 20% of data or minimum 10 points for evaluation
        eval_size = max(min(len(self.data) // 5, 50), 10)
        actual_prices = self.data['Close'].tail(eval_size).values

        for model_name, model in self.models.items():
            if model is None:
                continue

            try:
                if model_name == 'ARIMA':
                    fitted_values = model.fittedvalues.tail(eval_size).values
                elif model_name == 'SARIMA':
                    fitted_values = model.fittedvalues.tail(eval_size).values
                elif model_name == 'Prophet':
                    # Get fitted values from Prophet
                    df = model['data']
                    prophet_model = model['model']
                    fitted = prophet_model.predict(df)
                    fitted_values = fitted['yhat'].tail(eval_size).values
                elif model_name == 'LSTM':
                    # Evaluate LSTM on available data
                    X, y, scaler = self.prepare_lstm_data(model['lookback'])
                    if X is not None and len(X) > 0:
                        # Use last portion for evaluation
                        eval_start = max(0, len(X) - eval_size)
                        X_eval = X[eval_start:]
                        y_eval = y[eval_start:]

                        predictions = model['model'].predict(X_eval, verbose=0)
                        fitted_values = scaler.inverse_transform(predictions).flatten()
                        actual_prices = scaler.inverse_transform(y_eval.reshape(-1, 1)).flatten()
                    else:
                        continue

                # Ensure arrays are same length
                min_len = min(len(actual_prices), len(fitted_values))
                actual_prices_eval = actual_prices[-min_len:]
                fitted_values_eval = fitted_values[-min_len:]

                # Calculate metrics
                mape = mean_absolute_percentage_error(actual_prices_eval, fitted_values_eval) * 100
                rmse = np.sqrt(mean_squared_error(actual_prices_eval, fitted_values_eval))

                metrics[model_name] = {
                    'MAPE': round(mape, 2),
                    'RMSE': round(rmse, 2)
                }

                print(f"✅ {model_name}: MAPE={mape:.2f}%, RMSE={rmse:.2f}")

            except Exception as e:
                print(f"❌ {model_name} evaluation failed: {e}")
                metrics[model_name] = {'MAPE': 'N/A', 'RMSE': 'N/A'}

        self.metrics = metrics
        return metrics

    def plot_comprehensive_results(self, days=30):
        """Create comprehensive visualization with error handling"""
        try:
            if self.data is None or self.data.empty:
                print("❌ No data available for plotting")
                return

            fig = make_subplots(
                rows=2, cols=2,
                subplot_titles=(
                    'Historical Data & Forecasts',
                    'Model Performance (MAPE %)',
                    'Forecast Comparison',
                    'Price Trend Analysis'
                ),
                specs=[[{"colspan": 2}, None],
                       [{"type": "bar"}, {"type": "scatter"}]]
            )

            # Historical data
            dates = self.data.index
            fig.add_trace(
                go.Scatter(x=dates, y=self.data['Close'],
                          name='Historical Price', line=dict(color='blue', width=2)),
                row=1, col=1
            )

            # Forecasts
            colors = {'ARIMA': 'red', 'SARIMA': 'green', 'Prophet': 'purple', 'LSTM': 'orange'}

            if self.forecasts:
                future_dates = pd.date_range(start=dates[-1] + pd.Timedelta(days=1), periods=days)

                for model_name, forecast in self.forecasts.items():
                    if forecast is not None and len(forecast) > 0:
                        fig.add_trace(
                            go.Scatter(x=future_dates, y=forecast,
                                      name=f'{model_name} Forecast',
                                      line=dict(color=colors.get(model_name, 'black'),
                                              dash='dash', width=2)),
                            row=1, col=1
                        )

            # Model performance bar chart
            if self.metrics:
                models = list(self.metrics.keys())
                mapes = []
                for m in models:
                    mape_val = self.metrics[m]['MAPE']
                    mapes.append(mape_val if mape_val != 'N/A' else 0)

                if any(m > 0 for m in mapes):
                    fig.add_trace(
                        go.Bar(x=models, y=mapes, name='MAPE (%)',
                              marker_color=['lightblue', 'lightgreen', 'lightcoral', 'lightyellow'][:len(models)]),
                        row=2, col=1
                    )

            # Forecast values comparison
            if self.forecasts:
                for model_name, forecast in self.forecasts.items():
                    if forecast is not None and len(forecast) > 0:
                        fig.add_trace(
                            go.Scatter(x=list(range(1, len(forecast)+1)), y=forecast,
                                      name=f'{model_name} Trend', mode='lines+markers',
                                      line=dict(color=colors.get(model_name, 'black'))),
                            row=2, col=2
                        )

            fig.update_layout(
                height=800,
                title_text=f"Complete Stock Analysis: {self.symbol}",
                showlegend=True
            )
            fig.show()

        except Exception as e:
            print(f"❌ Plotting failed: {e}")

    def generate_comprehensive_report(self):
        """Generate detailed analysis report with error handling"""
        try:
            if self.data is None or self.data.empty:
                return "❌ No data available for report generation"

            current_price = self.data['Close'].iloc[-1]

            report = f"""
📈 COMPREHENSIVE STOCK MARKET ANALYSIS REPORT
{'='*60}

🏢 STOCK INFORMATION:
- Symbol: {self.symbol}
- Current Price: ${current_price:.2f}
- Analysis Period: {self.period}
- Data Points: {len(self.data)} records
- Date Range: {self.data.index[0].strftime('%Y-%m-%d')} to {self.data.index[-1].strftime('%Y-%m-%d')}

🤖 MODELS IMPLEMENTED:
"""

            for model_name in ['ARIMA', 'SARIMA', 'Prophet', 'LSTM']:
                status = "✅ Trained" if model_name in self.models and self.models[model_name] is not None else "❌ Failed"
                report += f"        - {model_name}: {status}\n"

            if self.metrics:
                report += f"""
📊 MODEL PERFORMANCE METRICS:
"""

                for model_name, metrics in self.metrics.items():
                    report += f"        - {model_name}: MAPE={metrics['MAPE']}%, RMSE=${metrics['RMSE']}\n"

            if self.forecasts:
                report += f"""
🔮 FORECAST SUMMARY (30 days):
"""
                for model_name, forecast in self.forecasts.items():
                    if forecast is not None and len(forecast) > 0:
                        forecast_price = forecast[-1]
                        change = ((forecast_price - current_price) / current_price) * 100
                        report += f"        - {model_name}: ${forecast_price:.2f} ({change:+.2f}%)\n"

            # Find best model
            best_model = "N/A"
            best_mape = float('inf')

            for model_name, metrics in self.metrics.items():
                if metrics['MAPE'] != 'N/A' and metrics['MAPE'] < best_mape:
                    best_mape = metrics['MAPE']
                    best_model = model_name

            # Calculate trend and volatility safely
            try:
                trend = 'Upward' if current_price > self.data['Close'].iloc[-min(30, len(self.data)-1)] else 'Downward'
                volatility = (self.data['Close'].std() / self.data['Close'].mean()) * 100
                avg_volume = self.data['Volume'].tail(min(10, len(self.data))).mean() / 1000000
            except:
                trend = 'Unknown'
                volatility = 0
                avg_volume = 0

            report += f"""
🏆 BEST PERFORMING MODEL: {best_model} (MAPE: {best_mape}%)

📋 TECHNICAL ANALYSIS SUMMARY:
- Trend: {trend}
- Volatility: {volatility:.2f}%
- Volume Trend: {avg_volume:.2f}M avg (last 10 days)

⚠️  DISCLAIMER:
This analysis is for educational purposes only.
Not financial advice. Past performance doesn't guarantee future results.
Always consult with financial professionals before making investment decisions.
"""

            return report

        except Exception as e:
            return f"❌ Report generation failed: {e}"

# MAIN EXECUTION
if __name__ == "__main__":
    print("🚀 STARTING COMPLETE STOCK MARKET ANALYSIS...")
    print("=" * 60)

    # Initialize forecaster
    forecaster = CompleteStockForecaster('AAPL', '2y')

    # Step 1: Data Collection & Analysis
    print("\n📊 STEP 1: DATA COLLECTION")
    data = forecaster.fetch_data()

    if data is None:
        print("❌ Failed to fetch data. Exiting...")
        exit()

    print("\n🔍 STEP 2: TIME SERIES ANALYSIS")
    decomposition = forecaster.analyze_time_series()

    # Step 2: Train All Models
    print("\n🤖 STEP 3: TRAINING ALL MODELS")
    print("-" * 40)

    # Train each model
    forecaster.train_arima(order=(2,1,2))
    forecaster.train_sarima(order=(1,1,1), seasonal_order=(1,1,1,7))
    forecaster.train_prophet()
    forecaster.train_lstm(epochs=20)

    # Step 3: Generate Forecasts
    print("\n🔮 STEP 4: GENERATING FORECASTS")
    forecasts = forecaster.forecast_all_models(days=30)

    # Step 4: Evaluate Models
    print("\n📊 STEP 5: EVALUATING MODELS")
    metrics = forecaster.evaluate_models()

    # Step 5: Visualize Results
    print("\n📈 STEP 6: CREATING VISUALIZATIONS")
    forecaster.plot_comprehensive_results()

    # Step 6: Generate Report
    print("\n📋 STEP 7: GENERATING REPORT")
    report = forecaster.generate_comprehensive_report()
    print(report)

    # Save results
    try:
        if forecasts:
            results_data = []
            for model_name in forecasts.keys():
                forecast_val = forecasts[model_name][-1] if len(forecasts[model_name]) > 0 else None
                mape_val = metrics.get(model_name, {}).get('MAPE', 'N/A')
                rmse_val = metrics.get(model_name, {}).get('RMSE', 'N/A')

                results_data.append({
                    'Model': model_name,
                    'Forecast_30_days': forecast_val,
                    'MAPE': mape_val,
                    'RMSE': rmse_val
                })

            results_df = pd.DataFrame(results_data)
            filename = f'{forecaster.symbol}_complete_analysis.csv'
            results_df.to_csv(filename, index=False)
            print(f"\n✅ Complete analysis saved to {filename}")

        print("\n🎉 ALL ANALYSIS COMPLETE!")
        print("\n📦 DELIVERABLES GENERATED:")
        print("   ✓ Historical data analysis")
        print("   ✓ ARIMA model implementation")
        print("   ✓ SARIMA model implementation")
        print("   ✓ Prophet model implementation")
        print("   ✓ LSTM model implementation")
        print("   ✓ Model comparison & evaluation")
        print("   ✓ Interactive visualizations")
        print("   ✓ Comprehensive report")
        print("   ✓ CSV results export")

    except Exception as e:
        print(f"❌ Error saving results: {e}")
        print("✅ Analysis completed but results not saved")

🚀 STARTING COMPLETE STOCK MARKET ANALYSIS...

📊 STEP 1: DATA COLLECTION
📈 Fetching data for AAPL...
✅ Data fetched: 500 records
📅 Date range: 2023-09-05 to 2025-09-02

🔍 STEP 2: TIME SERIES ANALYSIS

🔍 TIME SERIES ANALYSIS:
📊 Stationarity Test (ADF):
   - ADF Statistic: -1.8534
   - p-value: 0.3543
   - Is Stationary: No
📈 Seasonal decomposition completed with period=14

🤖 STEP 3: TRAINING ALL MODELS
----------------------------------------

🔄 Training ARIMA(2, 1, 2) model...
✅ ARIMA model trained successfully!
   - AIC: 2672.70

🔄 Training SARIMA(1, 1, 1)x(1, 1, 1, 7) model...


DEBUG:cmdstanpy:input tempfile: /tmp/tmpyz6m1t_o/rkwnpoma.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpyz6m1t_o/bb5qeqxl.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=55754', 'data', 'file=/tmp/tmpyz6m1t_o/rkwnpoma.json', 'init=/tmp/tmpyz6m1t_o/bb5qeqxl.json', 'output', 'file=/tmp/tmpyz6m1t_o/prophet_modelwcnr381p/prophet_model-20250902145204.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:52:04 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:52:04 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


✅ SARIMA model trained successfully!
   - AIC: 2625.00

🔄 Training Prophet model...
✅ Prophet model trained successfully!

🔄 Training LSTM model...
✅ LSTM model trained successfully!
   - Final Loss: 0.008241

🔮 STEP 4: GENERATING FORECASTS

🔮 Generating 30-day forecasts...
✅ ARIMA forecast generated
✅ SARIMA forecast generated
✅ Prophet forecast generated
✅ LSTM forecast generated

📊 STEP 5: EVALUATING MODELS

📊 EVALUATING MODEL PERFORMANCE...
✅ ARIMA: MAPE=0.94%, RMSE=2.87
✅ SARIMA: MAPE=0.94%, RMSE=2.96
✅ Prophet: MAPE=4.51%, RMSE=12.70
✅ LSTM: MAPE=2.07%, RMSE=6.50

📈 STEP 6: CREATING VISUALIZATIONS



📋 STEP 7: GENERATING REPORT

📈 COMPREHENSIVE STOCK MARKET ANALYSIS REPORT

🏢 STOCK INFORMATION:
- Symbol: AAPL
- Current Price: $228.62
- Analysis Period: 2y
- Data Points: 500 records
- Date Range: 2023-09-05 to 2025-09-02

🤖 MODELS IMPLEMENTED:
        - ARIMA: ✅ Trained
        - SARIMA: ✅ Trained
        - Prophet: ✅ Trained
        - LSTM: ✅ Trained

📊 MODEL PERFORMANCE METRICS:
        - ARIMA: MAPE=0.94%, RMSE=$2.87
        - SARIMA: MAPE=0.94%, RMSE=$2.96
        - Prophet: MAPE=4.51%, RMSE=$12.7
        - LSTM: MAPE=2.07%, RMSE=$6.5

🔮 FORECAST SUMMARY (30 days):
        - ARIMA: $227.97 (-0.29%)
        - SARIMA: $231.33 (+1.18%)
        - Prophet: $206.52 (-9.67%)
        - LSTM: $227.31 (-0.58%)

🏆 BEST PERFORMING MODEL: ARIMA (MAPE: 0.94%)

📋 TECHNICAL ANALYSIS SUMMARY:
- Trend: Upward
- Volatility: 11.57%
- Volume Trend: 36.06M avg (last 10 days)

⚠️  DISCLAIMER: 
This analysis is for educational purposes only. 
Not financial advice. Past performance doesn't guarantee fu